In [73]:
import torch
from torch.utils.data import DataLoader, TensorDataset
import torch.nn as nn
import torch.nn.functional as F
import math

In [74]:
class SafeTensor(torch.Tensor):
    def __new__(cls, data: torch.Tensor, shift: float = None, eps: float = 1e-8):
        # Создаем объект как подкласс
        obj = torch.Tensor._make_subclass(cls, data.clone().detach())
        
        # Вычисляем параметры сдвига
        abs_data = data.abs()
        min_val = data.min().item()
        min_positive = abs_data[abs_data > 0].min().item() if (abs_data > 0).any() else 1.0
        computed_shift = abs(min_val) + min_positive

        # Инициализируем атрибуты объекта
        obj._eps = eps
        obj._shift = shift if shift is not None else computed_shift
        obj._logdata = torch.log2(data.abs() + obj._shift + eps)
        
        return obj

    def _inverse_transform(self):
        return torch.exp2(self._logdata) - self._shift

    def data(self):
        return self._inverse_transform()

    def __repr__(self):
        return f"SafeTensor({self.data().__repr__()}, shift={self._shift:.4f})"

    def clone(self):
        return SafeTensor(self.data(), self._shift, self._eps)

    def to(self, *args, **kwargs):
        return SafeTensor(self.data().to(*args, **kwargs), self._shift, self._eps)

    @property
    def log_repr(self):
        return self._logdata

In [75]:
class SafeLinearFunction(torch.autograd.Function):
    @staticmethod
    def forward(ctx, input_log, weight_log, bias_log, shift, eps):
        # Восстанавливаем оригинальные значения
        input_real = torch.exp2(input_log) - shift
        weight_real = torch.exp2(weight_log)
        bias_real = torch.exp2(bias_log)
        
        # Линейная операция
        output_real = F.linear(input_real, weight_real, bias_real)
        
        # Возвращаем в лог-пространстве
        output_log = torch.log2(output_real.abs() + shift + eps)
        return output_log

    @staticmethod
    def backward(ctx, grad_output_log):
        # Здесь должна быть реализация backward, но для простоты пока вернем None
        return None, None, None, None, None

class SafeLinear(nn.Module):
    def __init__(self, in_features, out_features, shift=1.0, eps=1e-8):
        super().__init__()
        self.in_features = in_features
        self.out_features = out_features
        self.shift = shift
        self.eps = eps
        
        # Инициализация параметров в логарифмическом пространстве
        weight_real = torch.randn(out_features, in_features) * 0.01
        bias_real = torch.zeros(out_features)
        
        self.weight_log = nn.Parameter(torch.log2(weight_real.abs() + eps))
        self.bias_log = nn.Parameter(torch.log2(bias_real.abs() + eps))
    
    def forward(self, input_log):
        return SafeLinearFunction.apply(
            input_log, 
            self.weight_log,
            self.bias_log,
            self.shift,
            self.eps
        )
    
    def extra_repr(self):
        return f'in_features={self.in_features}, out_features={self.out_features}, shift={self.shift:.4f}'

In [76]:
class SafeMLP(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super().__init__()
        self.fc1 = SafeLinear(input_dim, hidden_dim)
        self.fc2 = SafeLinear(hidden_dim, output_dim)

    def forward(self, x: SafeTensor):
        x = F.relu(self.fc1(x).data())  # real tensor to relu
        return self.fc2(SafeTensor(x)).data()  # финальный real output

In [77]:
def train(model, dataloader, epochs=10, lr=0.01):
    optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
    criterion = nn.CrossEntropyLoss()
    
    model.train()
    for epoch in range(epochs):
        total_loss = 0
        for i, (inputs, targets) in enumerate(dataloader):
            # Прямой проход
            optimizer.zero_grad()
            safe_inputs = SafeTensor(inputs)
            outputs = model(safe_inputs)
            loss = criterion(outputs, targets)
            
            # Обратный проход
            loss.backward()
            # Шаг оптимизации
            optimizer.step()
            total_loss += loss.item()
        print(f"Эпоха {epoch+1}/{epochs}, Loss: {total_loss:.4f}")


def get_synthetic_data(n_samples=1024, input_dim=784, num_classes=10):
    X = torch.randn(n_samples, input_dim)
    y = torch.randint(0, num_classes, (n_samples,))
    return TensorDataset(X, y)

In [78]:
dataset = get_synthetic_data()
dataloader = DataLoader(dataset, batch_size=64, shuffle=True)
model = SafeMLP(input_dim=784, hidden_dim=128, output_dim=10)
# Обучение с защитой от битфлипов
train(model, dataloader, epochs=100)

AttributeError: 'SafeTensor' object has no attribute '_logdata'